In [2]:
import tensorflow as tf
import numpy as np
#from chiron.chiron_model import read_config
#from chiron.chiron_model import inference
from google.protobuf import text_format

/home/heavens/anaconda3/envs/tensorflow-1.10.1/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def input_output_list():
    x = tf.placeholder(tf.float32, shape=[FLAGS.batch_size, FLAGS.segment_len])
    seq_length = tf.placeholder(tf.int32, shape=[FLAGS.batch_size])
    training = tf.placeholder(tf.bool)
    config_path = os.path.join(FLAGS.model,'model.json')
    model_configure = read_config(config_path)    
    logits, _ = inference(x, 
                          seq_length, 
                          training=training,
                          full_sequence_len = FLAGS.segment_len,
                          configure = model_configure)
    predict = tf.nn.ctc_greedy_decoder(tf.transpose(logits, perm=[1, 0, 2]), seq_length, merge_repeated=True)
    input_dict = {'x': x, 'seq_length': seq_length, 'training': training}
    output_dict = {'decoded_indices': predict[0][0].indices, 'decoded_values': predict[0][0].values,
                   'neg_sum_logits': predict[1]}
    return input_dict, output_dict

In [13]:
import tensorflow as tf
import sys
from tensorflow.python.platform import gfile

from tensorflow.core.protobuf import saved_model_pb2
from tensorflow.python.util import compat

input_path = "/home/heavens/Chiron_project/Chiron/chiron/model/chiron_dna/1/saved_model.pb"
log_path = "/home/heavens/Chiron_project/Chiron/chiron/model/chiron_dna/inference/"
graph_def = tf.GraphDef()
with tf.Session() as sess:
    with open(input_path,"rb") as f:
        data = compat.as_bytes(f.read())
        sm = saved_model_pb2.SavedModel()
        sm.ParseFromString(data)
        #print(sm)
        if 1 != len(sm.meta_graphs):
            print('More than one graph found. Not sure which to write')
            sys.exit(1)

        #graph_def = tf.GraphDef()
        #graph_def.ParseFromString(sm.meta_graphs[0])
        tf.import_graph_def(sm.meta_graphs[0].graph_def)
        sub_graph = tf.graph_util.extract_sub_graph(sm.meta_graphs[0].graph_def,["CTCBeamSearchDecoder"])
        train_writer = tf.summary.FileWriter(log_path)
        tf.import_graph_def(sub_graph)
        train_writer.add_graph(sess.graph)
        train_writer.flush()
        train_writer.close()
        sess.close()

In [11]:
tf.import_graph_def(sm.meta_graphs[0].graph_def)

In [13]:
sub_graph

node {
  name: "Placeholder"
  op: "Placeholder"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: 1100
        }
        dim {
          size: 1000
        }
      }
    }
  }
}
node {
  name: "Placeholder_1"
  op: "Placeholder"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: 1100
        }
      }
    }
  }
}
node {
  name: "Reshape/shape"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
          dim {
            size: 4
          }
        }
        tensor_content: "L\004\000\000\001\000\000\000\350\003\000\000\001\000\000\000"
      }
    }
  }
}
node {
  name: "Reshape"
  op: "Reshape"
  input: "Placeholder"
  input: "Reshape/shape"
  attr {
    k

In [10]:

import tensorflow as tf
from tensorflow.python.platform import gfile
GRAPH_EXP_PATH = '/home/heavens/Chiron_project/Chiron/chiron/model/DNA_default/export/1'
GRAPH_PB_PATH = GRAPH_EXP_PATH+'saved_model.pb'
with tf.Session() as sess:
    graph_def = tf.saved_model.loader.load(sess, [tf.saved_model.tag_constants.SERVING], GRAPH_EXP_PATH)
    print("load graph")
#    with gfile.FastGFile(GRAPH_PB_PATH,'rb') as f:
#       graph_def = tf.GraphDef()
#       text_format.Merge(f.read(),graph_def)
    sess.graph.as_default()
    tf.train.import_meta_graph(graph_def)
    graph_nodes=[n for n in graph_def.node]
    names = []
    for t in graph_nodes:
      names.append(t.name)
    print(names)


INFO:tensorflow:Restoring parameters from /home/heavens/Chiron_project/Chiron/chiron/model/DNA_default/export/1/variables/variables
load graph


AttributeError: 'MetaGraphDef' object has no attribute 'node'

In [7]:
tf.contrib.util.make_tensor_proto(np.array([1,2,3,4]),shape=[4])

dtype: DT_INT64
tensor_shape {
  dim {
    size: 4
  }
}
tensor_content: "\001\000\000\000\000\000\000\000\002\000\000\000\000\000\000\000\003\000\000\000\000\000\000\000\004\000\000\000\000\000\000\000"

In [15]:
graph_def.ListFields()

[(<google.protobuf.pyext._message.FieldDescriptor at 0x7fb927d17e30>,
  stripped_op_list {
    op {
      name: "Add"
      input_arg {
        name: "x"
        type_attr: "T"
      }
      input_arg {
        name: "y"
        type_attr: "T"
      }
      output_arg {
        name: "z"
        type_attr: "T"
      }
      attr {
        name: "T"
        type: "type"
        allowed_values {
          list {
            type: DT_BFLOAT16
            type: DT_HALF
            type: DT_FLOAT
            type: DT_DOUBLE
            type: DT_UINT8
            type: DT_INT8
            type: DT_INT16
            type: DT_INT32
            type: DT_INT64
            type: DT_COMPLEX64
            type: DT_COMPLEX128
            type: DT_STRING
          }
        }
      }
    }
    op {
      name: "All"
      input_arg {
        name: "input"
        type: DT_BOOL
      }
      input_arg {
        name: "reduction_indices"
        type_attr: "Tidx"
      }
      output_arg {
        name